# Web Crawler Implementation on TAU courses syllabus

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import dateutil.parser as dparser

### using beautiful soup

In [9]:
page = requests.get('http://yedion.tau.ac.il/yed/yednew.dll?MfcISAPICommand=but&year=2014&course_num=03654150')
soup = BeautifulSoup(page.content, 'html.parser')

tables = soup.findAll("table", {"class" : "printtable"})
records = [] # store all of the records in this list
for table in tables:
    rec = []
    heads = table.findAll('th')
    rows = table.findAll('tr', {"dir" : "rtl"})
    for h in heads:
        rec.append(h.contents[0])
    for d in rows[0].findAll('td'):
        rec.append(d.contents[0])
    records.append(rec)
  
for i in records: 
    for j in i:
        print j,
    print

# tabulka = soup.find("table", {"class" : "printtable"})


אלגוריתמי קירוב לאופטימיזציה קומבינטורית 0365-4150-01 מדעים מדויקים ב' 1200 - 0900 ה' 209  שרייבר מתמטי שיעור ד"ר בוכבינדר ניב


### Parse to df

In [10]:
def parse_to_df_html(html_add):
    df = pd.read_html(html_add)
    df = df[0].dropna(how = 'all')
    df = df.reset_index()
    df.drop('index',axis=1, inplace=True)
    
    if df.shape[0] == 1:
        return
    # Create the answer df
    data = pd.DataFrame(columns=('LastUpdate', 'CourseName', 'CourseNumber', 'Faculty','LecturerName','ClassType','Building','Room','Day','Hour','Semester'))

    #convert df to a dictionary
    dic = df.to_dict()

    #static field - lastUpdate
    lastUpdate = dparser.parse(dic[0][0],fuzzy=True).strftime('%m/%d/%Y')

    #looping over each course
    i = 1
    j= 0
    while i< df.shape[0]:
        k=1
        while i+k < df.shape[0]:
            if pd.isnull(df.loc[i+k,1]) == False and contain_courseNum(df.loc[i+k,1]):
                break
            k +=1

        main = df.loc[i+2,]
        data.loc[j] = [lastUpdate,dic[0][i],dic[1][i],dic[0][i+1],main[6],main[5], main[4], main[3], main[2], main[1], main[0]]
        j +=1
        i = i+k
    return data

In [11]:
def contain_courseNum(num):    
    return len([i for i, ltr in enumerate(num) if ltr == '-']) ==2

# MAIN

In [ ]:
# get course links
page = requests.get('https://engineering.tau.ac.il/yedion/2015-16/14')
soup = BeautifulSoup(page.content, 'html.parser')
divs = soup.find('div', {'id':'block-system-main'})
courses = set()
for a in divs.findAll('a', {'class':None}):
    href =  a.attrs['href']
    if href[:23]=='http://yedion.tau.ac.il':
        courses.add(href)
        
# create a df for all cuorses
ans = pd.DataFrame()

for i in courses:
    toAdd = parse_to_df_html(i)
    ans = pd.concat([ans, toAdd])

In [ ]:
data = parse_to_df_html('http://yedion.tau.ac.il/yed/yednew.dll?MfcISAPICommand=but&year=2015&course_num=05714177')

In [ ]:
data